In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.899558,"""a""",0.229664,0.256526,0.72187,0.311084,0.050011,-0.971374,0.07316
0.841471,1,"""a""",0,0.064726,"""a""",0.664453,0.180569,0.075196,0.180145,0.288093,0.041051,0.160187
0.909297,2,"""a""",1,0.674234,"""a""",0.153421,0.87613,0.123643,0.453647,0.828232,0.100404,0.401917
0.14112,3,"""a""",0,0.874555,"""a""",0.43337,0.201315,0.602938,0.395077,0.047534,-0.77896,0.086648
-0.756802,4,"""a""",0,0.792694,"""a""",0.712802,0.224694,0.850838,0.121831,0.368446,-1.101926,0.108726


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.311084,-0.050011
-0.841471,-0.180145,-0.288093
-0.909297,-0.453647,-0.828232
-0.14112,-0.395077,-0.047534
0.756802,0.189253,-0.318436


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.490971, -0.335858]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.490971, -0.335858]","[0.087308, 0.40765]"


In [9]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.218074,0.001667,130.806007,0.0,0.214806,0.221342
"""exp(x2)""",0.174725,0.000677,258.168478,0.0,0.173398,0.176051
"""sin(x3)""",-1.744711,0.00134,-1302.056079,0.0,-1.747337,-1.742084
"""__bias__""",-0.107372,0.001516,-70.832979,0.0,-0.110344,-0.104401


In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.503307, -0.324541]"
"""a""","[-0.503307, -0.324541]"
"""a""","[-0.503307, -0.324541]"
"""a""","[-0.503307, -0.324541]"
"""a""","[-0.503307, -0.324541]"
…,…
"""b""","[-0.478507, -0.347092]"
"""b""","[-0.478507, -0.347092]"
"""b""","[-0.478507, -0.347092]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.229664,0.256526,-0.971374,-0.198915,-0.772459
0.664453,0.180569,0.041051,-0.386873,0.427924
0.153421,0.87613,0.100404,-0.36958,0.469985
0.43337,0.201315,-0.77896,-0.280385,-0.498575
0.712802,0.224694,-1.101926,-0.42543,-0.676496


In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.503307, -0.324541]"
"""b""","[-0.478507, -0.347092]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[-0.304714, -0.17975]"
"""a""","[-0.33981, -0.146104]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""b""",-0.539791
"""a""",-0.538295


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.971374,0.229664,0.256526,null,null
0.041051,0.664453,0.180569,null,null
0.100404,0.153421,0.87613,null,null
-0.77896,0.43337,0.201315,null,null
-1.101926,0.712802,0.224694,"[-1.103068, 0.063002]",-0.772113
…,…,…,…,…
-0.926431,0.210114,0.672296,"[-0.268079, -0.359585]",-0.298074
-1.001942,0.01259,0.472897,"[-0.64989, -0.738418]",-0.357378
-0.537548,0.307926,0.798006,"[-0.538165, -1.123762]",-1.062484


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.225434, 28.892064, 28.547297]"


In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
29.018661,"[-0.555539, 0.831491]"
28.553195,"[0.831491, 0.555539]"


In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.275683
-0.004978
0.292201
-0.324403
0.094231


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.332056,0.994231,0.506202,0.500792,0.503483,0.506771,0.503406
"""b""",0.333237,1.00294,0.50768,0.512118,0.509889,0.505433,0.501286


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""church"""
"""to"""
"""world"""
"""going"""
"""hello"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
""""""
"""go"""
"""church"""
"""hello"""
"""world"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.717466
-0.988356
-0.849425


In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.245882,null
null,2.345723,null
-0.717466,0.793501,-0.127803
-0.988356,1.79947,-1.646556
-0.849425,-0.273934,0.640647


In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""l""",null
null,"""QgzFo""",null
-0.717466,"""Up""","""0nO"""
-0.988356,"""kS65r""","""6l"""
-0.849425,"""Mn""","""s"""


In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.717466,"""5upaX"""
-0.988356,"""6WlSO"""
-0.849425,"""CNZp4"""


In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-1.217326,null,-1.217346
null,-2.222686,null,-2.222747
-0.717466,0.391669,3.075768,0.391931
-0.988356,1.797652,2.057802,1.797659
-0.849425,-1.606459,1.248237,-1.60674


In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.195184,-0.806495,8
null,0.390904,0.201841,7
-0.717466,0.841611,0.074251,3
-0.988356,0.507716,0.476893,6
-0.849425,0.931553,0.160644,4


In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.806014,0.420364,1.109708,0.574156


In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.098887,0.417608,3,3
1,0.623386,0.336108,1,0
2,0.820118,0.472125,3,3
0,0.993241,0.301729,0,7
1,0.150876,0.573866,0,4


In [40]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.126213,0.899566}","{27.723629,0.836904}","{1.835628,0.119041}"


In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.106546,0.915156}  ┆ {39.978362,0.297851} ┆ {1.779589,0.130319} │
│ 1         ┆ {0.842022,0.399836}  ┆ {46.090936,0.120931} ┆ {0.452691,0.770494} │
│ 2         ┆ {-1.154271,0.248472} ┆ {28.413107,0.812156} ┆ {1.50678,0.197672}  │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
554,0.1108
571,0.1142
548,0.1096
554,0.1108
552,0.1104
558,0.1116
582,0.1164
519,0.1038
562,0.1124


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.121245,0.47607,0.673376,0.065312,8.640811,16
1,0.285398,0.151423,0.226397,0.882159,3.988109,22
2,0.03445,0.764929,0.4883,0.884881,4.579601,11
3,0.363656,0.060313,0.30585,0.098741,5.460819,17
4,0.465217,0.261634,0.98319,0.928263,6.522145,11


In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.121245,0.47607,0.673376,0.065312,8.640811,1
1,0.285398,0.151423,0.226397,0.882159,3.988109,816
2,0.03445,0.764929,0.4883,0.884881,4.579601,698
3,0.363656,0.060313,0.30585,0.098741,5.460819,1
4,0.465217,0.261634,0.98319,0.928263,6.522145,818


In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.121245,0.47607,0.673376,0.065312,8.640811,"[0, 1933, … 1723]"
1,0.285398,0.151423,0.226397,0.882159,3.988109,"[1, 1368, … 1501]"
2,0.03445,0.764929,0.4883,0.884881,4.579601,"[2, 703, … 1245]"
3,0.363656,0.060313,0.30585,0.098741,5.460819,"[3, 595, … 1641]"
4,0.465217,0.261634,0.98319,0.928263,6.522145,"[4, 1750, … 1124]"


In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1933, … 1562] ┆ 8                  │
│ 1   ┆ [1, 1368, … 910]  ┆ 6                  │
│ 2   ┆ [2, 703, … 96]    ┆ 7                  │
│ 3   ┆ [3, 595, … 1067]  ┆ 8                  │
│ 4   ┆ [4, 1750, … 787]  ┆ 6                  │
└─────┴───────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.121245,0.47607,0.673376,0.065312,8.640811,"[0, 1933, … 1723]","[0.0, 0.053175, … 0.065791]"
1,0.285398,0.151423,0.226397,0.882159,3.988109,"[1, 1368, … 1501]","[0.0, 0.036398, … 0.076018]"
2,0.03445,0.764929,0.4883,0.884881,4.579601,"[2, 703, … 1245]","[0.0, 0.061503, … 0.074099]"
3,0.363656,0.060313,0.30585,0.098741,5.460819,"[3, 595, … 1641]","[0.0, 0.073428, … 0.076157]"
4,0.465217,0.261634,0.98319,0.928263,6.522145,"[4, 1750, … 1124]","[0.0, 0.052367, … 0.089612]"


In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.121245,0.47607,0.673376,0.065312,8.640811
11,0.435831,0.532263,0.647762,0.950091,7.263676
15,0.452773,0.178854,0.618258,0.492514,2.527533
16,0.579769,0.37267,0.432255,0.495162,5.715883
18,0.22862,0.442084,0.297393,0.529558,4.086089


In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
11,0.435831,0.532263,0.647762,0.950091,7.263676
66,0.555439,0.564487,0.584126,0.228685,1.064785
108,0.540756,0.497996,0.407182,0.92444,7.896406
115,0.531247,0.494733,0.800553,0.153616,6.703229
123,0.560505,0.485626,0.233371,0.637934,9.77159


In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
108,0.540756,0.497996,0.407182,0.92444,7.896406
115,0.531247,0.494733,0.800553,0.153616,6.703229
123,0.560505,0.485626,0.233371,0.637934,9.77159
157,0.476479,0.483291,0.395204,0.359498,6.865205
168,0.49928,0.499163,0.80711,0.360983,0.601422


In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1634, 526]",3
1,"[1, 390, 733]",3
2,"[2, 1190, 1023]",3
3,"[3, 1809, … 1561]",5
4,"[4, 1453, … 1350]",4


# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures.

In [53]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_7655/3354819425.py:3: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
0.0,0.141414,2,8,"""8""","""eb"""
0.0,0.537703,1,6,"""Ok""","""R3"""
0.0,0.328249,0,0,"""N""","""KO"""
0.0,0.76383,0,4,"""3V""","""S"""
0.0,0.206697,0,2,"""Dh""","""Lz"""


In [54]:
df_pd = df.to_pandas()

In [55]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

jaccard_col
f64
0.3


In [56]:
# Polars Series
print(pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest())
# Pyarrow Series
print(pds2.query_binary_metrics(df["actual"].to_arrow(), df["predicted"].to_arrow()).struct.unnest())
# NumPy
print(pds2.query_binary_metrics(df["actual"].to_numpy(), df["predicted"].to_numpy()).struct.unnest())

shape: (1, 5)
┌───────────┬─────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall  ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---     ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64     ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪═════════╪══════════╪═══════════════════╪══════════╡
│ 0.497808  ┆ 0.49735 ┆ 0.497579 ┆ 0.497842          ┆ 0.497038 │
└───────────┴─────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬─────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall  ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---     ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64     ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪═════════╪══════════╪═══════════════════╪══════════╡
│ 0.497808  ┆ 0.49735 ┆ 0.497579 ┆ 0.497842          ┆ 0.497038 │
└───────────┴─────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬─────────┬──────────┬

In [57]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.189757,0.2,0.178,0.002564
0.386246,0.2,0.192,0.000327
0.582808,0.2,0.214,0.000947
0.798914,0.2,0.227,0.003419
inf,0.2,0.189,0.000622


In [58]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

cid_ce
f64
13.245407


In [59]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

c3_stats
f64
0.132187


In [60]:
pds2.str_leven(df["s1"], df["s2"]).head() 

str_leven
u32
2
2
2
2
2
2
2
1
1
